In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from utils import calculate_cvar_left, Expando
from Barter_Set import Barter_Set

# Constants
HOURS = 2
DAYS = 2
SCENARIOS = 1000


class Barter_Visualization():
    def __init__(self):
        np.random.seed(42)  # You can choose any integer as seed
    
        T = HOURS * DAYS
        self.data= Expando()
        self.data.price_true = np.random.normal(7, 3, size=(T, SCENARIOS))
        self.data.prod = np.random.normal(10, 5, size=(T, SCENARIOS)) # Capacity 300
        self.data.demand = np.random.normal(20, 10, size=(T, SCENARIOS))
        self.data.correlation = np.corrcoef(self.data.price_true,self.data.prod)[0,1]
        self.data.retail_price = 8
        self.dS = -1 
        self.data.alpha = 0.95
        self.data.strikeprice_min = 3
        self.data.strikeprice_max = 10
        self.data.contract_amount_min = 0 
        self.data.contract_amount_max = 20
        self.data.cost=0

    def setup_statistics(self):

         # Biased price views
        bias_G = self.data.K_G * self.data.price_true.mean()
        bias_L = self.data.K_L * self.data.price_true.mean()
        self.data.price_G = self.data.price_true + bias_G
        self.data.price_L = self.data.price_true + bias_L
        # Threat Points for generator and Load 
      
        num_scenarios = self.data.price_true.shape[1]  # Match your existing number of scenarios
        time_periods = self.data.price_true.shape[0]   # Match your existing time periods
            
        self.data.lambda_sum_true_per_scenario = self.data.price_true.sum(axis=0) # Sum across time periods for each scenario
        # Calculate E^P(λ∑) - Expected value of the sum over T (using TRUE distribution)
        self.data.expected_lambda_sum_true = self.data.lambda_sum_true_per_scenario.mean()
        
        # Capture Prices and Capture Rate 

        #Capture_price = self.data.net_earnings_no_contract_true_G / self.data.prod.sum(axis=0)  # Capture price for G
        #Capture_rate = Capture_price / self.data.price_true.mean(axis=0)  # Capture rate for G (should it be the total mean price or the mean price per scenario?)

        # Average Capture Price and Rate 
        #Capture_price_avg = Capture_price.mean()
        #Capture_rate_avg = Capture_rate.mean()

        # Calculate CVaR^P(λ∑) - CVaR of the sum over T (using TRUE distribution)
        # Assumes calculate_cvar returns the expected value of the variable *given* it's in the alpha-tail
        self.data.left_cvar_lambda_sum_true = calculate_cvar_left(self.data.lambda_sum_true_per_scenario, self.data.alpha)
        #elf.data.left_Cvar_capture_lambda_sum_true = calculate_cvar_left(Capture_rate*self.data.lambda_sum_true_per_scenario, self.data.alpha) * Capture_rate_avg

        # Calculate CVaR^P(-λ∑) - CVaR of the negative sum over T (using TRUE distribution)
        # This corresponds to the risk of low LMPs
        self.data.left_cvar_neg_lambda_sum_true = calculate_cvar_left(-self.data.lambda_sum_true_per_scenario, self.data.alpha)
        #self.data.left_Cvar_neg_capture_lambda_sum_true = calculate_cvar_left(-Capture_rate*self.data.lambda_sum_true_per_scenario, self.data.alpha) * Capture_rate_avg


        # Determine the *absolute* bias constants K_G and K_L for the equations
        # Based on Theorem 1, K_G/K_L are absolute shifts.:
        self.data.K_G_lambda_Sigma = self.data.K_G * self.data.expected_lambda_sum_true
        self.data.K_L_lambda_Sigma = self.data.K_L * self.data.expected_lambda_sum_true


        # New Objective function defition 
        
        
        self.data.term1_G_new =(( (1-self.data.A_G) * self.data.expected_lambda_sum_true +self.data.K_G_lambda_Sigma ) - self.data.A_G * self.data.left_cvar_neg_lambda_sum_true)/ time_periods     # SR* numerator for Gen

        # high-price risk (feeds S_U*)
        self.data.term2_G_new = ( ( (1-self.data.A_G) * self.data.expected_lambda_sum_true + self.data.K_G_lambda_Sigma ) + self.data.A_G * self.data.left_cvar_lambda_sum_true) / time_periods     # SU* numerator for Gen
  
        # Load-serving entity ––––––––––––––––––––––––––––––––––––––––––––––
        # low-price risk  (feeds S_R*)
        self.data.term3_L_SR_new = ( self.data.expected_lambda_sum_true
                        + self.data.A_L * self.data.left_cvar_lambda_sum_true
                        + self.data.K_L_lambda_Sigma - self.data.A_L * self.data.expected_lambda_sum_true  ) / time_periods   # SU* numerator for LSE
        # high-price risk (feeds S_U*)
        self.data.term4_L_SU_new = ( self.data.expected_lambda_sum_true
                        - self.data.A_L * self.data.left_cvar_neg_lambda_sum_true
                        + self.data.K_L_lambda_Sigma  - self.data.A_L * self.data.expected_lambda_sum_true )  / time_periods   # SR* numerator for LSE




         # Calculate SR* using Equation (27) - Minimum of the relevant terms
        self.data.SR_star = np.min([self.data.term1_G_new, self.data.term2_G_new,self.data.term3_L_SR_new])

        # Calculate SU* using Equation (28) - Maximum of the relevant terms
        self.data.SU_star = np.max([self.data.term1_G_new, self.data.term2_G_new,self.data.term4_L_SU_new])

        
        # Print calculated critical bounds for verification during runtime
        print(f"Calculated SR* using New (Eq 27): {self.data.SR_star:.5f}")
        print(f"Calculated SU* using new (Eq 28): {self.data.SU_star:.5f}")
        print()

        if self.data.SR_star < self.data.strikeprice_min:
            self.data.strikeprice_min = self.data.SR_star-1
        if self.data.SU_star > self.data.strikeprice_max:
            self.data.strikeprice_max = self.data.SU_star+1
       

     
    # Function to calculate CVaR (left-tail)
    def Utility_G(self, strike,volume):

        """
        strike : Strike Price [float]
        volume : Contract volume [float]
        """
        rev_contract = (volume * (strike - self.data.price_G)).sum(axis=0)
        Expected = self.data.net_earnings_no_contract_priceG_G 
        earnings = Expected + rev_contract
        CVaR_G = calculate_cvar_left(earnings, self.data.alpha)
    
        Utility = (1-self.data.A_G)*earnings.mean() + self.data.A_G * CVaR_G
        return Utility

    def Utility_L(self, strike,volume):
        rev_contract = (volume * (self.data.price_L - strike)).sum(axis=0)
        Expected = self.data.net_earnings_no_contract_priceL_L
        earnings = Expected + rev_contract
        CVaR_L = calculate_cvar_left(earnings, self.data.alpha)
        
      
        Utility =(1-self.data.A_L)*earnings.mean() + self.data.A_L * CVaR_L
        return Utility
    


    def Condition_lemma5_MR(self):
        n = 100
        cvars_list = np.zeros(n)

        M_space = np.linspace(self.data.contract_amount_min,self.data.contract_amount_min+1 , n)
        
        for i in range (len(M_space)):
            rev_contract = (M_space[i] * (self.data.strikeprice_min- self.data.price_L )).sum(axis=0)
            Expected = self.data.net_earnings_no_contract_priceL_L
            earnings = Expected + rev_contract
            cvars_list[i] = calculate_cvar_left(earnings, self.data.alpha)

        dx = M_space[1] - M_space[0]  # Step size
        gradient = np.gradient(cvars_list,M_space,edge_order=2)  
        slope = gradient[M_space == self.data.contract_amount_min]
        #If condition is true, then the condition is satisfied, and there exists a barter set 
        rhs = ((self.data.A_G*self.data.left_cvar_lambda_sum_true)/self.data.A_L - (self.data.A_G * self.data.expected_lambda_sum_true)/self.data.A_L 
            +self.data.expected_lambda_sum_true + self.data.K_G_lambda_Sigma/self.data.A_L - self.data.K_L_lambda_Sigma/self.data.A_L) 
        cond = slope < rhs
        print(f"Condition Lemma 5 MR: {cond}")
        return cond

    def Condition_lemma5_MU(self):

        n = 100
        cvars_list = np.zeros(n)
        M_space = np.linspace(self.data.contract_amount_max,self.data.contract_amount_max-1 , n)
        
        for i in range (len(M_space)):
            rev_contract = (M_space[i] * ( self.data.strikeprice_min - self.data.price_L)).sum(axis=0)
            Expected = self.data.net_earnings_no_contract_priceL_L
            earnings = Expected + rev_contract
            cvars_list[i] = calculate_cvar_left(earnings, self.data.alpha)

        dx = M_space[1] - M_space[0]  # Step size
        gradient = np.gradient(cvars_list,M_space,edge_order=2)  

        slope = gradient[M_space == self.data.contract_amount_max]
        #If condition is true, then the condition is satisfied, and there exists a barter set 
        rhs = (-(self.data.A_G*self.data.left_cvar_neg_lambda_sum_true)/self.data.A_L - (self.data.A_G * self.data.expected_lambda_sum_true)/self.data.A_L 
            +self.data.expected_lambda_sum_true + self.data.K_G_lambda_Sigma/self.data.A_L - self.data.K_L_lambda_Sigma/self.data.A_L) 
        cond = slope > rhs
        print(f"Condition Lemma 5 MU: {cond}")

        return cond

    def calculate_utility_derivative(self, M_space, V_1_Low, V_2_High):
            # Initial slope calculations

            n = len(M_space)

            dx = M_space[1] - M_space[0]  # Step size
            duG_1 = np.gradient(V_1_Low[:,0],M_space,edge_order=2)  
            duL_1 = np.gradient(V_1_Low[:,1],M_space,edge_order=2)  
            slope_1 = duL_1 / duG_1  
            
            duG_2 = np.gradient(V_2_High[:,0],M_space,edge_order=2)
            duL_2 = np.gradient(V_2_High[:,1],M_space,edge_order=2)
            slope_2 = duL_2 / duG_2

            if slope_1[0] < self.dS:
                cond_MR = True
            else:
                cond_MR = False
            
            if slope_1[-1] > self.dS:
                cond_MU = True
            else:
                cond_MU = False

            if cond_MR == False:
                print("No Barter Set exists, as the conditions of Lemma 5 are not satisfied.")
                M_SR,M_SU = 0,0
                return cond_MR,cond_MU,None, None, M_SR, M_SU ,None , None
            elif cond_MR == True and cond_MU == False:
                print("Barter Set exists, no concave part in the utility curves")
                M_SR,M_SU = self.data.contract_amount_max, self.data.contract_amount_max
                return cond_MR,cond_MU,None, None, M_SR, M_SU ,None , None
            else :
                print("Barter Set exists, concave part in the utility curves")

      
            # Find first crossing points
            mask_negative_v1 = slope_1 >= self.dS
            mask_positive_v2 = slope_2 <= self.dS
            first_index_negative_v1 = np.argmax(mask_negative_v1)
            first_index_positive_v2 = np.argmax(mask_positive_v2)
            M_SR = M_space[first_index_negative_v1]
            M_SU = M_space[first_index_positive_v2]

          
            return cond_MR,cond_MU,slope_1, slope_2, M_SR, M_SU ,first_index_negative_v1 , first_index_positive_v2

    # Simulation + Plotting Function
    def run_visualization(self,M,Capture,A_G=0.5, A_L=0.5, K_G=0.0, K_L=0.0,zoom = False):
        # Generate synthetic data
        self.data.CR = Capture
        self.data.A_G = A_G
        self.data.A_L = A_L
        self.data.K_G = K_G
        self.data.K_L = K_L
      

        self.setup_statistics()
        n = 1000

        self.data.net_earnings_no_contract_G_df= self.data.prod*(self.data.price_true-self.data.cost)
        # True net earnings for G given correct price distribution
        self.data.net_earnings_no_contract_true_G = self.data.net_earnings_no_contract_G_df.sum(axis=0) # Generator G earnings in numpy array for calculations ( Total Sum per Scenario)
        # Biased net earnings for G given biased price distribution K
        self.data.net_earnings_no_contract_priceG_G_df = Capture*self.data.prod*self.data.price_G
        self.data.net_earnings_no_contract_priceG_G =  self.data.net_earnings_no_contract_priceG_G_df.sum(axis=0)
        # Load L earnings in numpy array for calculations
        self.data.net_earnings_no_contract_true_L = (self.data.demand*(self.data.retail_price-self.data.price_true)).sum(axis=0)
        # Load L earnings in numpy array for calculations
        self.data.net_earnings_no_contract_priceL_L = (self.data.demand*(self.data.retail_price-self.data.price_L)).sum(axis=0)
        
        #CVaR calculation of No contract (TRue )
        self.data.CVaR_no_contract_priceG_G = calculate_cvar_left(self.data.net_earnings_no_contract_priceG_G,self.data.alpha) # CVaR for G
        self.data.CVaR_no_contract_priceL_L = calculate_cvar_left(self.data.net_earnings_no_contract_priceL_L,self.data.alpha)

        self.data.Zeta_G=(1-self.data.A_G)*self.data.net_earnings_no_contract_priceG_G.mean() + self.data.A_G*self.data.CVaR_no_contract_priceG_G
        self.data.Zeta_L=(1-self.data.A_L)*self.data.net_earnings_no_contract_priceL_L.mean() + self.data.A_L*self.data.CVaR_no_contract_priceL_L



        V_1_Low= np.zeros((n,2))
        V_2_High = np.zeros((n,2))
        M_space = np.linspace(self.data.contract_amount_min, self.data.contract_amount_max, n)

        threat_point_x = self.data.Zeta_G
        threat_point_y = self.data.Zeta_L


        # Calculate the utility for each contract revenue
        for i in range(len(M_space)):            #Curve 1 
            V_1_Low[i,0] = self.Utility_G(self.data.strikeprice_min, M_space[i]) 
            V_1_Low[i,1] = self.Utility_L(self.data.strikeprice_min, M_space[i])
            #Curve 2
            V_2_High[i,0] = self.Utility_G(self.data.strikeprice_max, M_space[i]) 
            V_2_High[i,1] = self.Utility_L(self.data.strikeprice_max, M_space[i])

        
        cond_MR,cond_MU,slope_1, slope_2, M_SR,M_SU, first_index_v1,first_index_v2= self.calculate_utility_derivative(M_space,V_1_Low, V_2_High)
        # Calculate the slope of the utility curves     

        #Calculate Utlity for the optimal contract amount        
        UG_Low_Mopt = self.Utility_G(self.data.strikeprice_min, M_SR)
        UL_Low_Mopt = self.Utility_L(self.data.strikeprice_min, M_SR)
        UG_High_Mopt = self.Utility_G(self.data.strikeprice_max, M_SU)
        UL_High_Mopt = self.Utility_L(self.data.strikeprice_max, M_SU)
        UG_Low_Mopt_SR = self.Utility_G(self.data.SR_star, M_SR)
        UL_Low_Mopt_SR = self.Utility_L(self.data.SR_star, M_SR)
        UG_High_Mopt_SU = self.Utility_G(self.data.SU_star, M_SU)
        UL_High_Mopt_SU = self.Utility_L(self.data.SU_star,M_SU)

        UG_Low_Vis = self.Utility_G(self.data.strikeprice_min, M)
        UL_Low_Vis = self.Utility_L(self.data.strikeprice_min, M)
        UG_High_Vis = self.Utility_G(self.data.strikeprice_max, M)
        UL_High_Vis = self.Utility_L(self.data.strikeprice_max, M)

     # Temporary test values 
        UG_term1 = self.Utility_G(self.data.term1_G_new,M_SR)
        UL_term1 = self.Utility_L(self.data.term1_G_new,M_SR)

        UG_term2 = self.Utility_G(self.data.term2_G_new,M_SR)
        UL_term2 = self.Utility_L(self.data.term2_G_new,M_SR)

        UG_term3 = self.Utility_G(self.data.term3_L_SR_new,M_SR)
        UL_term3 = self.Utility_L(self.data.term3_L_SR_new,M_SR)

        UG_term4 = self.Utility_G(self.data.term4_L_SU_new,M_SR)
        UL_term4 = self.Utility_L(self.data.term4_L_SU_new,M_SR)

        if cond_MR == True:
            slope_opt = np.round((UL_High_Mopt - UL_Low_Mopt) / (UG_High_Mopt - UG_Low_Mopt),0)
            b_opt = UL_Low_Mopt - slope_opt * UG_Low_Mopt
            vertical_intersect_y = slope_opt * threat_point_x + b_opt
            
            # Horizontal line intersection (y = threat_point_y)
            horizontal_intersect_x = (threat_point_y - b_opt) / slope_opt
    
        
        #self.plot_utility_cvar_vs_M(strike_price='min')
        #self.plot_utility_cvar_vs_M(strike_price='max')

        # Keeping SR constant and plotting through MR to MU (Curve 1)
        plt.figure(figsize=(10, 6))
        plt.plot(V_1_Low[:,0], V_1_Low[:,1], label='Curve 1', color='blue')
        plt.plot(V_2_High[:,0], V_2_High[:,1], label='Curve 2', color='red')

        arrow_positions = np.linspace(0,1,10)  # Positions along the curve (as fractions)
        for pos in arrow_positions:
            point_idx = int(len(V_1_Low) * pos)
            if point_idx + 1 < len(V_1_Low):
                plt.annotate('', 
                    xy=(V_1_Low[point_idx+1,0], V_1_Low[point_idx+1,1]),
                    xytext=(V_1_Low[point_idx,0], V_1_Low[point_idx,1]),
                    arrowprops=dict(arrowstyle='->', color='blue', lw=2),
                    annotation_clip=True)

        # Add multiple direction arrows for Curve 2 (red)
        for pos in arrow_positions:
            point_idx = int(len(V_2_High) * pos)
            if point_idx + 1 < len(V_2_High):
                plt.annotate('', 
                    xy=(V_2_High[point_idx+1,0], V_2_High[point_idx+1,1]),
                    xytext=(V_2_High[point_idx,0], V_2_High[point_idx,1]),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2),
                    annotation_clip=True)

        # Add single "M increasing" label for each curve
        plt.annotate('Curve 1 : M increasing', 
                    xy=(V_1_Low[len(V_1_Low)//2,0], V_1_Low[len(V_1_Low)//2,1]),
                    xytext=(30, 30), textcoords='offset points',
                    color='blue', fontsize=10)
        plt.annotate('Curve 2 : M increasing',
                    xy=(V_2_High[len(V_2_High)//2,0], V_2_High[len(V_2_High)//2,1]),
                    xytext=(30, 30), textcoords='offset points',
                    color='red', fontsize=10)

        # Hopefully this should be at the intersection points 
        # Plot the points
        
        if cond_MR==True:
            plt.scatter(UG_Low_Mopt_SR, UL_Low_Mopt_SR, color='cyan', marker='D', s=100, 
            label=f'SR Utility (M={M_SR:.2f})')
            plt.scatter(UG_High_Mopt_SU, UL_High_Mopt_SU, color='magenta', marker='D', s=100, 
            label=f'SU Utility (M={M_SU:.2f})')

            # Plot Optimal Contract Amount Point with fixed price SR and SU
            plt.scatter(UG_Low_Mopt, UL_Low_Mopt, color='green', marker='o', s=100, label=f'V1 M* = ({M_SR:2f} MW)')
            plt.scatter(UG_High_Mopt, UL_High_Mopt, color='green', marker='*', s=150, label=f'V2 M* = ({M_SU:2f} MW)')

            # Draw straight line between optimal points
            plt.plot([UG_Low_Mopt, UG_High_Mopt], 
                    [UL_Low_Mopt, UL_High_Mopt], 
                'g--',
                label='Line between optimal points',
                alpha=0.3)  # alpha makes line slightly transparent        
            plt.plot([UG_Low_Vis, UG_High_Vis],
                    [UL_Low_Vis, UL_High_Vis], 
                'o--', 
                alpha=0.7)
            plt.scatter(UG_Low_Vis, UL_Low_Vis, color='orange', marker='o', s=100, label=f'V1 M = ({M:2f} MW)')
            plt.scatter(UG_High_Vis, UL_High_Vis, color='orange', marker='*', s=150, label=f'V2 M = ({M:2f} MW)')


            # Plot Nash bargaining terms
            plt.scatter(UG_term1, UL_term1, color='brown', marker='s', s=100, label='Term 1 (G)')
            plt.scatter(UG_term2, UL_term2, color='brown', marker='v', s=100, label='Term 2 (G)')
            plt.scatter(UG_term3, UL_term3, color='purple', marker='s', s=100, label='Term 3 (L)')
            plt.scatter(UG_term4, UL_term4, color='purple', marker='v', s=100, label='Term 4 (L)')

            # Plot Utility G and L for the contract at optimal solution 
            #plt.scatter(self.results.utility_G, self.results.utility_L, color='orange', marker='o', s=100, label='Optimal Solution (G,L)')
            #Plot scipy resulting utility 
            #plt.scatter(self.scipy_results.utility_G, self.scipy_results.utility_L, color='red', marker='o', s=100, label='Scipy Result (G,L)')

            vertices = np.array([
            [threat_point_x, threat_point_y],  # Start at threat point
            [threat_point_x, vertical_intersect_y],      # Go right to first optimal point
            [horizontal_intersect_x, threat_point_y],        # Go to second optimal point      # Go down vertically
            [threat_point_x, threat_point_y]    # Back to start
            ])

            # Create and add the polygon
            polygon = plt.Polygon(vertices, facecolor='gray', alpha=0.2, edgecolor=None)
            plt.gca().add_patch(polygon)

               # Plot intersection points
            plt.scatter(threat_point_x, vertical_intersect_y, 
                color='purple', marker='x', s=100,) 
                #label='Vertical Line Intersection')
            plt.scatter(horizontal_intersect_x, threat_point_y, 
                color='purple', marker='x', s=100,) 
                #label='Horizontal Line Intersection')

        # After plotting the threat point, add horizontal and vertical lines
        # Add horizontal line from threat point
        plt.axhline(y=threat_point_y, color='black', linestyle='--', alpha=0.3)

        # Add vertical line from threat point  
        plt.axvline(x=threat_point_x, color='black', linestyle='--', alpha=0.3)

        # Original threat point plotting
        plt.scatter(threat_point_x, threat_point_y, color='black', marker='o', s=100, label='Threatpoint')


        plt.xlabel('Utility G')
        plt.ylabel('Utility L')

        # Set the x and y limits to zoom in
        if zoom:
           plt.xlim(threat_point_x - 200, UG_High_Mopt_SU+200)
           plt.ylim(threat_point_y - 1000, UL_Low_Mopt_SR )

        
  
        plt.title(r'Barter Set (1-A)*E($\pi$) + A*CVaR($\pi$)')
        plt.legend()
        plt.grid()
        plt.show()


BS_Vis = Barter_Visualization()

# Use the instance method `run_visualization` with widgets.interact
widgets.interact(
    BS_Vis.run_visualization,
     M =widgets.FloatSlider(value=10, min=0, max=60, step=5, description='Contract Amount'),
     Capture =widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='Capture Rate'),
    A_G=widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='A_G'),
    A_L=widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='A_L'),
    K_G=widgets.FloatSlider(value=0.0, min=-0.1, max=0.1, step=0.01, description='K_G'),
    K_L=widgets.FloatSlider(value=0.0, min=-0.1, max=0.1, step=0.01, description='K_L'),
    zoom = widgets.widgets.Checkbox(value=False, description="Zoom In")
)


interactive(children=(FloatSlider(value=10.0, description='Contract Amount', max=60.0, step=5.0), FloatSlider(…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>